In [ ]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master/')
from glob import glob
import pickle
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import random
import math

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

!pip install efficientunet-pytorch
from efficientunet import *
import timm
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.autograd import Function
from torchvision import transforms
import itertools

Air conditionor + GPU 
electric fee...

SETI test 5

In [ ]:
train_df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_df_old = pd.read_csv('../input/seti-breakthrough-listen/old_leaky_data/train_labels_old.csv')
test_df = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')

train_df['filepath'] = train_df.id.apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
train_df_old['filepath'] = train_df_old.id.apply(lambda x : f'../input/seti-breakthrough-listen/old_leaky_data/train_old/{x[0]}/{x}.npy')
test_df['filepath'] = test_df.id.apply(lambda x : f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')

In [ ]:
class setiDataset(Dataset):
    def __init__(self, df): 
        self.df = df
    
    def fileinfo(self, idx):
        return self.df.filepath.iloc[idx], self.df.target.iloc[idx]
    
    def __getitem__(self, idx):
        filepath, target = self.fileinfo(idx)
        image = np.load(filepath).astype('float32')
        image = np.stack([image[0], image[2], image[4]]).transpose(1,2,0)
        image = cv2.resize(image, (320,320), interpolation=cv2.INTER_LINEAR)
        
        image = ToTensorV2()(image=image)['image'] 
        return image, torch.tensor(target, dtype=torch.long)
    
    def __len__(self):
        return len(self.df)

In [ ]:
class new_old_Dataset(Dataset):
    def __init__(self, df, df_old): 
        N = min(len(df[df.target==1]), len(df_old[df_old.target==1]))
        self.df = df[df.target==1].sample(N).reset_index(drop=True)
        self.df_old = df_old[df_old.target==1].sample(N).reset_index(drop=True)
        
    def fileinfo(self, idx):
        return self.df.filepath.iloc[idx], self.df_old.filepath.iloc[idx]
    
    def __getitem__(self, idx):
        filepath, old_filepath = self.fileinfo(idx)
        image = np.load(filepath).astype('float32')
        image = np.stack([image[0], image[2], image[4]]).transpose(1,2,0)
        image = cv2.resize(image, (320,320), interpolation=cv2.INTER_LINEAR)
        image = ToTensorV2()(image=image)['image'] 
        
        old_image = np.load(old_filepath).astype('float32')
        old_image = np.stack([old_image[0], old_image[2], old_image[4]]).transpose(1,2,0)
        old_image = cv2.resize(old_image, (320,320), interpolation=cv2.INTER_LINEAR)
        old_image = ToTensorV2()(image=old_image)['image']
        
        return image, old_image
    
    def __len__(self):
        return len(self.df)

In [ ]:
class GradReverse(Function):
    @staticmethod
    def forward(ctx, x, lamda):
        ctx.lamda = lamda
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = (grad_output.neg() * ctx.lamda)
        return output, None

class domain_classifier(nn.Module):
    def __init__(self):
        super(domain_classifier, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 2, bias=False),
        )
        
    def forward(self, x, lam=0.0):
        x = GradReverse.apply(x, lam)
        return self.main(x).view(-1)

In [ ]:
def train(data_loader, 
              model, label_classifier, 
              optimizer, device):
    model.train()
    label_classifier.train()
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        images, targets = data
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        def closure():
            optimizer.zero_grad()
            output = model(images)
            output = label_classifier(output)
            loss = nn.BCEWithLogitsLoss()(output, targets.view(-1,1))
            loss.backward()
            return loss
        optimizer.step(closure)

In [ ]:
def adjust_lambda(pos, current_epoch, data_loader_size, total_epochs):
    p = (pos + current_epoch*data_loader_size)/(total_epochs*data_loader_size)
    lam = 2/(1+np.exp(-10*p)) - 1
    return lam

def train_dann(data_loader, epoch,
               model, label_classifier, domain_discriminator, 
               optimizer, optimizer_D, device):
    model.train()
    label_classifier.train()
    domain_discriminator.train()
    
    N = len(data_loader)
    for pos, data in tqdm(enumerate(data_loader), position=0, leave=True, total=N, desc='DANN Training'):
        images, images_old = data
        batch_size = images.size(0)
        targets = torch.ones(batch_size)
        targets_old = torch.zeros(batch_size)
        
        images = images.to(device, dtype=torch.float)
        images_old = images_old.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        targets_old = targets_old.to(device, dtype=torch.float)
        
        
        optimizer_D.zero_grad()
        optimizer.zero_grad()
        output = model(images)
        predict = label_classifier(output)
        loss = nn.BCEWithLogitsLoss()(predict, targets.view(-1,1))
        
        output_ = model(images_old)
        output = torch.cat([output,output_], dim=0) 
        lam = adjust_lambda(pos, epoch, N, total_epochs=5)   
        predict = domain_discriminator(output, lam)
        targets_old = torch.cat([targets, targets_old])
        loss += nn.BCEWithLogitsLoss()(predict, targets_old)
        loss.backward()
        
        optimizer.step()
        optimizer_D.step()

In [ ]:
def evaluate(data_loader, epoch, 
             model, label_classifier, device):
    model.eval()
    label_classifier.eval()
    final_targets = []
    final_outputs = []
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            images, targets = data

            images = images.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(images)
            output = label_classifier(output)
            
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
        
    roc_auc = metrics.roc_auc_score(final_targets, final_outputs)
    print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')

model = get_efficientunet_b0(out_channels=1, concat_input=True, pretrained=True)
label_classifier = timm.create_model('mobilenetv3_large_100', in_chans = 1, pretrained=True, num_classes=1)
domain_discriminator = domain_classifier()
model.to(device)
label_classifier.to(device)
domain_discriminator.to(device)

optimizer = torch.optim.Adam(list(model.parameters()) + list(label_classifier.parameters()), lr=5e-4, weight_decay=1e-5)
optimizer_D = torch.optim.Adam(domain_discriminator.parameters(), lr=5e-6, weight_decay=1e-7)

BATCH_C = 16
BATCH_D = 8

trn_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42)
trn_df = trn_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_dataset = setiDataset(trn_df)
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_C, shuffle=True)

valid_dataset = setiDataset(val_df)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_C, shuffle=False)

train_dataset_old = new_old_Dataset(trn_df, train_df_old)
train_loader_old= torch.utils.data.DataLoader(train_dataset_old, batch_size=BATCH_D, shuffle=True)

In [ ]:
for epoch in range(3):
    train(train_loader, 
              model, label_classifier, 
              optimizer, device
    )
    evaluate(valid_loader, epoch, 
             model, label_classifier, device
    )
    

for epcoh in range(3):
    train_dann(train_loader_old, epoch,
               model, label_classifier, domain_discriminator, 
               optimizer, optimizer_D, device
    )
    evaluate(valid_loader, epoch, 
             model, label_classifier, device
    )